# Think Bayes

Second Edition

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from empiricaldist import Pmf, Cdf
from utils import decorate, savefig

ModuleNotFoundError: No module named 'empiricaldist'

## Cumulative Distribution Functions

In [ ]:
from scipy.stats import binom

def update_binomial(pmf, data):
    """Update the PMF using the binomial distribution.
    
    pmf: Pmf representing the prior
    data: tuple of integers k and n
    """
    k, n = data
    xs = pmf.qs
    likelihood = binom.pmf(k, n, xs)
    pmf *= likelihood
    pmf.normalize()

In [ ]:
hypos = np.linspace(0, 1, 101)
pmf = Pmf(1, hypos)
data = 140, 250
update_binomial(pmf, data)

In [ ]:
cumulative = pmf.cumsum()

In [ ]:
def decorate_euro(title):
    decorate(xlabel='Proportion of heads (x)',
             ylabel='Probability',
             title=title)

In [ ]:
cumulative.plot(label='CDF')
pmf.plot(label='PMF')
decorate_euro(title='Posterior distribution for the Euro problem')
savefig('fig06-01')

In [ ]:
pmf[0.61]

In [ ]:
cumulative[0.61]

In [ ]:
from scipy.interpolate import interp1d

ps = cumulative.values
qs = cumulative.index

interp = interp1d(ps, qs)
interp(0.96)

In [ ]:
cdf = pmf.make_cdf()
cdf[0.61]

In [ ]:
try:
    cdf[0.615]
except KeyError as e:
    print('KeyError')

In [ ]:
cdf(0.615)

In [ ]:
cdf.quantile(0.9638303)

In [ ]:
cdf.credible_interval(0.9)

## Best Three of Four

In [ ]:
def make_die(sides):
    """Pmf that represents a die with the given number of sides.
    
    sides: int
    
    returns: Pmf
    """
    outcomes = np.arange(1, sides+1)
    die = Pmf(1/sides, outcomes)
    return die

In [ ]:
def add_dist_seq(seq):
    """Distribution of sum of values from PMFs.
    
    seq: sequence of Pmf objects
    
    returns: Pmf
    """
    total = seq[0]
    for other in seq[1:]:
        total = total.add_dist(other)
    return total

In [ ]:
die = make_die(6)
dice = [die] * 3

In [ ]:
pmf_3d6 = add_dist_seq(dice)

In [ ]:
def decorate_dice(title=''):
    decorate(xlabel='Outcome',
             ylabel='PMF',
             title=title)

In [ ]:
pmf_3d6.plot()
decorate_dice('Distribution of attributes')

In [ ]:
n = 10000
a = np.random.randint(1, 7, size=(n, 4))
a.sort(axis=1)
t = a[:, 1:].sum(axis=1)

In [ ]:
pmf_4d6 = Pmf.from_seq(t)

In [ ]:
pmf_3d6.plot(label='sum of 3 dice')
pmf_4d6.plot(label='best 3 of 4')

decorate_dice('Distribution of attributes')
savefig('fig06-02')

## Maximum

In [ ]:
from empiricaldist import Cdf

cdf_4d6 = pmf_4d6.make_cdf()
cdf_max6 = Cdf(cdf_4d6**6)

In [ ]:
pmf_max6 = cdf_max6.make_pmf()

In [ ]:
pmf_max6.plot(label='max of 6 attributes')

decorate_dice('Distribution of attributes')

In [ ]:
cdf_3d6 = pmf_3d6.make_cdf()
cdf_3d6.plot(label='best 3 of 4 dice')

cdf_4d6 = pmf_4d6.make_cdf()
cdf_4d6.plot(label='sum of 3 dice')

cdf_max6.plot(label='max of 6 attributes')

decorate_dice('Distribution of attributes')
plt.ylabel('CDF');

In [ ]:
cdf_max_dist6 = cdf_4d6.max_dist(6)

In [ ]:
np.max(np.abs(cdf_max_dist6 - cdf_max6))

## Minimum

Now let's compute the distribution for the minimum of your six attributes.

Remember that `Cdf(x)` is the probability that a value from the distribution is less than or equal to `x`.

So we can compute the probability that a value is greater than `x`, like this:

In [ ]:
prob_gt = 1 - cdf_4d6
prob_gt

Now here's the probability that all six values are greater than `x`:

In [ ]:
prob_gt6 = prob_gt**6
prob_gt6

And finally the probability that all six values are less than or equal to `x`.

In [ ]:
prob_le6 = 1-prob_gt6
prob_le6

The result is a Pandas Series that represents the CDF of the minimum of six attributes.  We can put those values in a `Cdf` object like this:

In [ ]:
cdf_min6 = Cdf(prob_le6)

Here's what it looks like.

In [ ]:
cdf_min6.plot(color='C2', label='minimum of 6')
cdf_max6.plot(color='C3', label='maximum of 6')
decorate_dice('Minimum and maximum of six attributes')
plt.ylabel('CDF')
savefig('fig06-03')

In [ ]:
cdf_min_dist6 = cdf_4d6.min_dist(6)

In [ ]:
np.max(np.abs(cdf_min_dist6 - cdf_min6))

## Mixture

In [ ]:
d4 = make_die(4)
d6 = make_die(6)

In [ ]:
total = Pmf.add(d4, d6, fill_value=0) / 2
total

In [ ]:
mix = Pmf(total)
mix.bar(alpha=0.7)
decorate_dice()

In [ ]:
total = Pmf.add(d4, 2*d6, fill_value=0) / 3
mix = Pmf(total)
mix.normalize()
mix.bar(alpha=0.7)
decorate_dice()

In [ ]:
hypos = [4,6,8]
counts = [1,2,3]
pmf_dice = Pmf(counts, hypos)
pmf_dice.normalize()
pmf_dice

In [ ]:
dice = [make_die(sides) for sides in hypos]

In [ ]:
product = dice * pmf_dice.ps
type(product)

In [ ]:
type(product[0])

In [ ]:
total = product[0]
for pmf in product[1:]:
    total = Pmf.add(total, pmf, fill_value=0)
    
total

In [ ]:
def make_mixture(pmf, pmf_seq):
    """Make a mixture of distributions.
    
    pmf: mapping from each hypothesis to its probability
    pmf_seq: sequence of Pmfs, each representing 
             a conditional distribution for one hypothesis
    """
    product = pmf_seq * pmf.ps
    total = product[0]
    for each_pmf in product[1:]:
        total = Pmf.add(total, each_pmf, fill_value=0)
    return Pmf(total)

In [ ]:
mix = make_mixture(pmf_dice, dice)
mix.bar(label='mixture', alpha=0.6)
decorate_dice('Mixture of Uniform Distributions')
savefig('fig06-04')

In [ ]:
mix.sum()

## Exercises

**Exercise:** When you generate a D&D character, instead of rolling dice, you can use the "standard array" of attributes, which is 15, 14, 13, 12, 10, and 8.

Do you think you are better off using the standard array or (literally) rolling the dice?

Compare the distribution of the values in the standard array to the distribution we computed for the best three out of four:

* Which distribution has higher mean?  Use the `mean` method.

* Which distribution has higher standard deviation?  Use the `std` method.

* The lowest value in the standard array is 8.  For each attribute, what is the probability of getting a value less than 8?  If you roll the dice six times, what's the probability that at least one of your attributes is less than 8?

* The highest value in the standard array is 15.  For each attribute, what is the probability of getting a value greater than 15?  If you roll the dice six times, what's the probability that at least one of your attributes is greater than 15?

To get you started, here's a `Cdf` that represents the distribution of attributes in the standard array:

In [ ]:
standard = [15,14,13,12,10,8]
cdf_standard = Cdf.from_seq(standard)

We can compare it to the distribution of attributes you get by rolling four dice at adding up the best three.

In [ ]:
cdf_4d6.plot(label='max of 6 attributes')
cdf_standard.step(label='standard set')

decorate_dice('Distribution of attributes')
plt.ylabel('CDF');

I plotted `cdf_standard` as a step function to show more clearly that it contains only a few values.

In [ ]:
# Solution

cdf_4d6.mean(), cdf_standard.mean()

In [ ]:
# Solution

cdf_4d6.std(), cdf_standard.std()

In [ ]:
# Solution

cdf_4d6.lt_dist(8)

In [ ]:
# Solution

cdf_4d6.gt_dist(15)

In [ ]:
# Solution

cdf_min6.lt_dist(8), 1 - (1-cdf_4d6.lt_dist(8))**6

In [ ]:
# Solution

cdf_max6.gt_dist(15), 1 - (1-cdf_4d6.gt_dist(15))**6

**Exercise:** Henri Poincaré was a French mathematician who taught at the Sorbonne around 1900. The following anecdote about him is probably fabricated, but it makes an interesting probability problem.

Supposedly Poincaré suspected that his local bakery was selling loaves of bread that were lighter than the advertised weight of 1 kg, so every day for a year he bought a loaf of bread, brought it home and weighed it. At the end of the year, he plotted the distribution of his measurements and showed that it fit a normal distribution with mean 950 g and standard deviation 50 g. He brought this evidence to the bread police, who gave the baker a warning.

For the next year, Poincaré continued the practice of weighing his bread every day. At the end of the year, he found that the average weight was 1000 g, just as it should be, but again he complained to the bread police, and this time they fined the baker.

Why? Because the shape of the distribution was asymmetric. Unlike the normal distribution, it was skewed to the right, which is consistent with the hypothesis that the baker was still making 950 g loaves, but deliberately giving Poincaré the heavier ones.

To see whether this anecdote is plausible, let's suppose that when the baker sees Poincaré coming, he hefts `n` loaves of bread and gives Poincaré the heaviest one.  How many loaves would the baker have to heft to make the average of the maximum 1000 g?

To get you started, I'll generate a year's worth of data from a normal distribution with the given parameters.

In [ ]:
mean = 950
std = 50
sample = np.random.normal(mean, std, size=365)

In [ ]:
# Solution

cdf = Cdf.from_seq(sample)

for n in range(2, 6):
    cdf_max = cdf.max_dist(n)
    print(n, cdf_max.mean())

In [ ]:
# Solution

cdf.plot(label='one loaf')
cdf.max_dist(4).plot(label='maximum of four loaves')

decorate(xlabel='Weight in grams',
         ylabel='CDF')

**Exercise:**  Suppose I have two boxes of dice:

* One contains a 4-sided die and a 6-sided die.

* The other contains a 6-sided die and an 8-sided die.

I choose a box at random, choose a die, and roll it 3 times.  If I get 2, 4, and 6, which box do you think I chose?

In [ ]:
# Solution

d4 = make_die(4)
d6 = make_die(6)
d8 = make_die(8)

In [ ]:
# Solution

pmf1 = Pmf(1/2, [4, 6])
mix1 = make_mixture(pmf1, [d4, d6])
mix1.bar()

In [ ]:
# Solution

pmf2 = Pmf(1/2, [6, 8])
mix2 = make_mixture(pmf2, [d6, d8])
mix2.bar(color='C1')

In [ ]:
# Solution

data = [2, 4, 6]
mix1(data)

In [ ]:
# Solution

mix2(data)

In [ ]:
# Solution

likelihood = [mix1(data).prod(), mix2(data).prod()]
likelihood

In [ ]:
# Solution

prior = Pmf(1/2, ['Box 1', 'Box 2'])
posterior = (prior * likelihood)
posterior.normalize()
posterior

**Exercise:** Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die.
I choose one of the dice at random, roll it, and report that the outcome is a 1.
If I roll the same die again, what is the probability that I get another 1?

Hint: Compute the posterior distribution as we have done before and pass it as one of the arguments to `make_mixture`.

In [ ]:
# Solution

hypos = [6, 8, 12]
prior = Pmf(1, hypos)
likelihood = 1/prior.qs
posterior = (prior * likelihood)
posterior.normalize()
posterior

In [ ]:
# Solution

d6 = make_die(6)
d8 = make_die(8)
d12 = make_die(12)

dice = d6, d8, d12

In [ ]:
# Solution

mix = make_mixture(posterior, dice)
mix.bar()

In [ ]:
# Solution

mix[1]